# Projeto 3 - Ciência dos Dados
___

#  Preditor de vitórias de lutas do UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Ortiz Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um modelo preditivo baseado em técnicas de aprendizado de máquina (*machine learning*) para prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil), os lutadores, seus históricos e outros dados da luta, a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 
### Assim esse projeto poderia ser usado para apostadores nas lutas de UFC e os próprios atlétas que participam nessas lutas, pois seria possível comparar os seus dados e os de seus oponentes, assim sabendo como está em relação a eles e quais fatores seria melhor treinar ou manter a fim de manter uma vantagem sobre eles.

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

____
## Método escolhido:






### Random Forest:
#### Esse método usa várias árvores de decisão para encontrar o que melhor se adequa, a que tem menos erros, para os nossos dados e o resultado que queremos, nesse caso quem é o vencedor.

![Diagrama Random Forest](randomforest.png)

[Link da imagem a acima](https://community.tibco.com/sites/default/files/styles/large/public/random_forest_diagram_0.png?itok=7vXTw5yz)



### Regressão Logística:
#### Esse método usa a função abaixo que vai sempre tender a 0 ou a 1, assim sendo um classificador binário. Ele atribui um coeficiente(β) para cada fator levado em consideração, assim tendo uma ordem de impacto dos fatores no resultado final.

$$Prob(y = 1 | X = x) = \frac{1}{1 + e^{-\left(\beta_0 + \beta_1 x_1 + \beta_2 x_2\right)}}$$

 ___
## Preparando o ambiente no jupyter:


### Imports:

In [ ]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


### Trabalhando com os Excels:

In [ ]:
data = pd.read_excel("data.xlsx")
data.head(2)

#### Sobre o nosso Dataframe

   O nosso dataframe consiste em uma coleção de dados de todas as lutas da história do UFC, desde 1993 até 2019, contando com profundas informações dos lutadores do evento, como o histórico do lutador, resultados das últimas lutas, idade e até dados de cada embate, por exemplo de quantos golpes foram desferidos na cabeça do adversário naquele específico combate.
   
   Após o dataframe ser lido e exibido pela biblioteca do Pandas, nós nos deparamos com uma tabela que separa as lutas pelas cores dos lados do ringue, sendo o "R_fighter" o lutador do lado vermelho e o "B_fighter" o oponente do lado azul e é assim que se divide as estatísticas das lutas em geral, no caso as informações do lado vermelho começam com o prefixo "R_" e segue o mesmo padrão com o lado azul ("B_").

In [ ]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

### Análise Exploratória

In [ ]:
print(data.Winner.value_counts())
ratio=3470/(3470+1591+83)
print('Porcentagem de vitórias dos lutadores do lado vermelho:',ratio*100,'%')

Como o objetivo do projeto é prever de qual lado/cor é o lutador vencedor do combate, fizemos uma contagem do número de vitórias de cada lado.

In [ ]:
plt.title("Red wins vs Blue wins")
data.Winner.value_counts().plot(kind='pie', colors=['red','blue','white'], autopct="%0.2f",legend=False);

Após plotar o gráfico da frequência relativa da vitória em seus respectivos lados, pudermos perceber que o lado vermelho acumula 67,46% das vitórias enquanto apenas 30,93% do lado azul, desbancando uma ideia inicial de igualdade entre os lados em relação às vitórias. 

Uma vez sabendo que os lutadores campeões lutam pelo lado vermelho, pensamos que talvez o lado vermelho poderia ter alguma vantagem na porcentagem de vitórias, já que geralmente os campeões são considerados melhores que a maioria de seus oponentes.
Portanto, resolvemos plotar um gráfico a fim de analisar esta vantagem e como esperado, os campeões realmente ganham mais que os desafiantes, cerca de uma proporção 4 para 1.

In [ ]:
graf_winner_with_title = data.loc[: , ["Winner", "title_bout"]]
graf_winner_with_title1 = graf_winner_with_title.loc[(graf_winner_with_title.title_bout==True),:]
plt.title("Red wins vs Blue wins em disputas pelo título")
graf_winner_with_title1.Winner.value_counts().plot(kind='pie', colors=['red','blue',"white"],autopct="%0.2f",legend=False);

Porém, quantas vezes ocorrem disputas por títulos de modo que possa alterar tão significantemente a porcentagem de vitórias dos lados?
Para isso, plotamos um gráfico da ocorrência relativa das disputas:

In [ ]:
plt.title("Porcentagem de lutas pelo título")
data['title_bout'].value_counts().plot(kind='pie', colors=['green','gold'],autopct="%0.2f",legend=False);

De todas as lutas que ocorreram no UFC, 7% são disputas por títulos, mostrando-se uma quantia pequena mas relativamente relevante em relação ao total.

Agora, para saber se esta vantagem se da apenas pelo fato do lado vermelho ser o campeão, decidimos fazer o mesmo gráfico de  vitórias do Red x vitórias do Blue mas apenas pelas lutas que não disputam o cinturão:

In [ ]:
graf_winner_with_title = data.loc[: , ["Winner", "title_bout"]]
graf_winner_with_title1 = graf_winner_with_title.loc[(graf_winner_with_title.title_bout==False),:]
plt.title("Red wins vs Blue wins sem disputa pelo título")
graf_winner_with_title1.Winner.value_counts().plot(kind='pie', colors=['red','blue',"white"],autopct="%0.2f",legend=False);

Como resultado, é possível analisar que mesmo em lutas quais não disputam o título, a vantagem do lado vermelho se mantém. Mas o que explicaria isso?

Após pesquisas, descobrimos que o lado vermelho, na maioria das vezes, é defendido por lutadores "favoritos" das apostas, ou seja, são lutadores apontados como os possíveis vencedores do confronto devido aos seus últimos resultados nos recentes embates e/ou o "status" que o lutador possui.

##### Mudando de assunto, queriamos saber como é distribuída as categorias de peso no UFC, para futuras análises...

In [ ]:
plt.figure(figsize=(20,15))
plt.title("Fighters distribuition by weight")
data['weight_class'].value_counts().plot(kind='pie',autopct="%0.2f",legend=False);

Após o plot do gráfico, resolvemos dividir as categorias por gênero:

In [ ]:
data_mulher = data.loc[(data.weight_class=="Women's Bantamweight")|
                       (data.weight_class=="Women's Featherweight")|
                       (data.weight_class=="Women's Flyweight")|
                       (data.weight_class=="Women's Strawweight"),:]
plt.title("Women's distribuition by weight")
data_mulher.weight_class.value_counts().plot(kind='pie',autopct="%0.2f",legend=False);

São 4 categorias de peso para as mulheres. Destacando-se entre os pesos galos ("Bantamweight") e palha ("Strawweight").

In [ ]:
data_homem = data.loc[(data.weight_class=="Flyweight")|
                       (data.weight_class=="Bantamweight")|
                       (data.weight_class=="Featherweight")|
                       (data.weight_class=="Lightweight")|
                       (data.weight_class=="Welterweight")|
                       (data.weight_class=="Middleweight")|
                       (data.weight_class=="Light Heavyweight")|
                       (data.weight_class=="Heavyweight"),:]
plt.title("Men's distribuition by weight")
data_homem.weight_class.value_counts().plot(kind='pie',autopct="%0.2f",legend=False);

São 8 categorias na divisão masculina, distribuídas majoritariamente entre os pesos leves ("Lightweights"), meio-médios ("Welterweight") e médios ("Middleweight"). 

Vamos ver se a superioridade do vermelho sobre o azul prevalece no gênero masculino...

In [ ]:
plt.title("Red wins vs Blue wins among men's")
data_homem.Winner.value_counts().plot(kind='pie', colors=['red','blue',"white"],autopct="%0.2f",legend=False);

Vamos ver se a superioridade do vermelho sobre o azul prevalece também no gênero feminino...

In [ ]:
plt.title("Red wins vs Blue wins among women's")
data_mulher.Winner.value_counts().plot(kind='pie', colors=['red','blue',"white"],autopct="%0.2f",legend=False);

Apesar de ser mais equilibrado, ainda prevalece a vantagem.

Por fim, vamos analisar se as categorias/pesos também afetam a quantidade de vitórias de cada lado.

Mas primeiro vamos fazer algumas funções...
A primeira, vamos determinar todos os pesos de qualquer divisão de gênero.
Enquanto na segunda vamos plotar os gráficos de vitórias entre os lados vermelhos e azuis para os homens.

In [ ]:
def pesos(df_pesos):
    data_homem_pesos = []
    for i in df_pesos:
        if i not in data_homem_pesos:
            data_homem_pesos.append(i)
    return data_homem_pesos

In [ ]:
def plota_graficos_por_peso(df):
    v=0
    i=-1
    for p in pesos(df.weight_class):
        v = data_homem.Winner.loc[(df.weight_class==p)]
        v.value_counts().plot(kind='pie',colors=['red','blue',"white"],autopct="%0.2f",legend=False);
        plt.title(p)
        plt.figure(i)
        i+=1
    return None

In [ ]:
plota_graficos_por_peso(data_homem)

Após a observação dos gráficos, é possivel concluir que o lado vermelho sempre leva vantagem, em todos os pesos e em alguns mais do que outros...

In [ ]:
#Escolhendo apenas as lutas entre lutadores da classe 'Heavyweight', pois as característica
#data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data['title_bout'] = data['title_bout'].map(bool_to_number)
data['Winner'] = data['Winner'].map(string_to_number)
data.head(2)

#### Blue = 0
#### Red = 1

In [ ]:
data_util = data.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [ ]:
data_util.head(2)

In [ ]:
data_util.dropna(inplace=True)
data_util.head(2)


### Tirando os dados categóricos: 

In [ ]:
#desconsiderando os dados categóricos para o teste
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_cat = data_util[categoricas].astype('category')
data_num = data_util.drop(categoricas, axis=1).astype('float')

___
# Teste 1
___

In [ ]:
X = data_num.drop('Winner', axis=1)
Y = data_num['Winner']


## Random Forest 1

### Separando os dados em testes e treinamento

In [ ]:
X_train_random, X_test_random, y_train_random, y_test_random = train_test_split(X, Y, test_size=0.25)

In [ ]:
model_random = RandomForestClassifier(n_estimators=10000)

model_random.fit(X_train_random, y_train_random)


### Verificando a performance 


In [ ]:
y_pred_random = model_random.predict(X_test_random)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_random, y_pred_random))

In [ ]:
y_test_random.value_counts(True)

In [ ]:
data_num.Winner.value_counts()

   ___
## Conclusão Random Forest 1 :

Tendo um acurácia média de 65% no primeiro modelo Random Forest não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 67.46%, se sempre afirmarmos que o vermelho é vencedor teremos uma acurácia maior.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para melhorar a acurácia.

## Regressão Logística 1

### Separando os dados em teste e treinamento

In [ ]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, Y, test_size=0.25)

In [ ]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [ ]:
model = LogisticRegression(max_iter=200000,solver='lbfgs', multi_class='auto')

model.fit(X_train_log, y_train_log)


### Verificando a performance 


In [ ]:
y_pred_log = model.predict(X_test_log)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_log, y_pred_log))

In [ ]:
y_test_random.value_counts(True)

In [ ]:
 data_num.Winner.value_counts()

In [ ]:
result = preparo(X_train_log,y_train_log)
#result.summary()

   ___
## Conclusão Regressão Logística 1 :

Tendo um acurácia média de 64% no primeiro modelo de Random Fores não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 67.46%, se sempre afirmarmos que o vermelho é o vencedor teremos uma acuracia maior.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para melhorar a acurácia.

___
## Melhor modelo da primeira iteração:  Random Forest (65%)

   ___
## Criando um dataframe dos fatores mais impactantes no resultado segundo o Teste 1:

In [ ]:
#Fatores que tem o maior peso na decisão da vitória no modelo Random Forest
j=1
lista_j=list()
for i, f in sorted(list(zip(model_random.feature_importances_, X_train_random.columns)), reverse=True):
    a=str(j)+'°'
    lista_j.append(a)
    j+=1

In [ ]:
data={'Fator':X_train_random.columns ,'Correlação':model_random.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head(2)

In [ ]:
plt.title('Correlation between the columns and the Winner column')
plt.scatter(Fator_por_corr.index,Fator_por_corr.Correlação, alpha=0.8)

   ___
## Escolhendo quais dados devem ser usados nos modelos de predição:

In [ ]:
def relevancia(df,coluna_nome,coluna_correlacao,acuracia):
    inuteis = []
    uteis = ['Winner']
    for index,row in df.iterrows():
        if row[coluna_correlacao] >= -acuracia and row[coluna_correlacao] <= acuracia:
            inuteis.append(row[coluna_nome])
        else:
            uteis.append(row[coluna_nome])
    return uteis

In [ ]:
uteis = relevancia(Fator_por_corr,'Fator','Correlação',0.01)

In [ ]:
data_util_relevante = data_util.loc[:,uteis]
data_util_relevante.head()

___
# Teste 2
____

In [ ]:
x = data_util_relevante.drop('Winner', axis=1)
y = data_util_relevante['Winner']

___
## Random Forest 2

### Separando os dados em testes e treinamento

In [ ]:
X_train_random2, X_test_random2, y_train_random2, y_test_random2 = train_test_split(x, y, test_size=0.25)

In [ ]:
model_random2 = RandomForestClassifier(n_estimators=10000)

model_random2.fit(X_train_random2, y_train_random2)

In [ ]:
#model_random2.feature_importances_

In [ ]:
y_pred_random2 = model_random2.predict(X_test_random2)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_random2, y_pred_random2))

In [ ]:
y_test_random2.value_counts(True)

In [ ]:
data_util_relevante.Winner.value_counts()


## Conclusão Random Forest 2

Após a retirada dos dados considerados inúteis pelo primeiro modelo de Random Forest a acurácia do modelo Random Forest subiu um pouco, antes com uma média de 65% e agora com uma de 68%, ainda é considerada baixa levando em consideração a maioria das vitórias pelo lado vermelho (67.46%). Se o modelo só tivesse como resultado vermelho a quantidade de acertos seria quase a mesma.

Para melhorar a acurácia pode-se diminuir ainda mais o número de variáveis que o modelo utiliza para prever o vencedor

___
## Montando a Regressão Logística 2



### Separando os dados em testes e treinamento

In [ ]:
X_train_log2, X_test_log2, y_train_log2, y_test_log2 = train_test_split(x, y, test_size=0.25)

In [ ]:
model = LogisticRegression(max_iter=500000,solver='lbfgs', multi_class='auto')

model.fit(X_train_log2, y_train_log2)

In [ ]:
y_pred_log2 = model.predict(X_test_log2)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_log2, y_pred_log2))

In [ ]:
y_test_log2.value_counts(True)

In [ ]:
data_util_relevante.Winner.value_counts()

In [ ]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [ ]:
result = preparo(x,y)
result.summary()

___
## Conclusão Regressão Logística 2

Após a retirada dos dados considerados inúteis pelo primeiro modelo a acurácia de ambos modelos subiu um pouco, antes com uma média de 64% e agora com uma de 66.5%, ainda é considerada baixa levando em consideração a maioria das vitórias pelo lado vermelho (67.46%). Se o modelo só tivesse como resultado a vitória do vermelho teriamos uma acurácia maior.

Para melhorar a acurâcia pode-se diminuir ainda mais o número de variáveis que o modelo usa para prever a vitória.

## Melhor modelo da segunda iteração: Random Forest (68%)

   ___
## Criando um dataframe dos fatores mais impactantes no resultado segundo o Teste 2:

In [ ]:
j2=1
lista_j2=list()
for i, f in sorted(list(zip(model_random2.feature_importances_, X_train_random2.columns)), reverse=True):
    
    a2=str(j2)+'°'
    lista_j2.append(a2)
    j2+=1


In [ ]:
data_mais_impor={'Fator':X_train_random2.columns ,'Correlação':model_random2.feature_importances_,}
Fator_por_corr2=pd.DataFrame(data_mais_impor)
Fator_por_corr2=Fator_por_corr2.sort_values(by='Correlação', ascending=False)
Fator_por_corr2['Grau de Importância']=lista_j2
Fator_por_corr2 = Fator_por_corr2.set_index('Grau de Importância')
Fator_por_corr2.head(2)

In [ ]:
uteis2 = relevancia(Fator_por_corr2,'Fator','Correlação',0.05)

In [ ]:
#uteis2

In [ ]:
data_util_relevante2 = data_util.loc[:,uteis2]
data_util_relevante2.head(2)

## Teste 3

In [ ]:
X3 = data_util_relevante2.drop('Winner', axis=1)
Y3 = data_util_relevante2['Winner']


## Random Forest 3

### Separando os dados em testes e treinamento

In [ ]:
X_train_random3, X_test_random3, y_train_random3, y_test_random3 = train_test_split(X3, Y3, test_size=0.25)

In [ ]:
model_random3 = RandomForestClassifier(n_estimators=10000)

model_random3.fit(X_train_random3, y_train_random3)

In [ ]:
#model_random3.feature_importances_

In [ ]:
y_pred_random3 = model_random3.predict(X_test_random3)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_random3, y_pred_random3))

In [ ]:
y_test_random3.value_counts(True)

In [ ]:
data_util_relevante2.Winner.value_counts()

___
## Conclusão Random Forest 3

Após a retirada dos dados que menos impactaram no resultado do Teste 2 a acurácia da terceira iteração do modelo Random Forest piorou: antes com uma média de acurácia de 68%, e agora com uma de 64%. 

Isso indica que para melhorar a acurácia não adianta reduzir o número de dados, provavelmete para melhorar a acurácia seria necessário separar os dados por classe de peso, ou sexo, ou até em faixas de tempo.

___
## Montando a Regressão Logística 3



### Separando os dados em testes e treinamento

In [ ]:
X_train_log3, X_test_log3, y_train_log3, y_test_log3 = train_test_split(X3, Y3, test_size=0.25)

In [ ]:
model_log3 = LogisticRegression(max_iter=500000,solver='lbfgs', multi_class='auto')

model_log3.fit(X_train_log3, y_train_log3)

In [ ]:
y_pred_log3 = model_log3.predict(X_test_log3)

In [ ]:
print('Acurácia do modelo:',accuracy_score(y_test_log3, y_pred_log3))

In [ ]:
y_test_log3.value_counts(True)

In [ ]:
data_util_relevante2.Winner.value_counts()

In [ ]:
result = preparo(X3,Y3)
result.summary()

___
## Conclusão Regressão Logística 3

Após a retirada dos dados que menos impactaram no resultado do Teste 2 a acurácia do modelo Regressão Logística piorou, antes com uma média de 66.5%, e agora com uma de 65%.

Isso indica que para melhorar a acurácia não adianta reduzir o número de dados, provavelmete para melhorar a acurácia seria necessário separar os dados por classe de peso, ou sexo, ou até em faixas de tempo.

___
## Melhor modelo da terceira iteração: Regressão Logística (65%)

# Naive-Bayes:

___
# Crie a Sua Luta - **Em desenvolvimento**

Com o objetivo de poder escolher dois nomes para o algoritimo checar quem provavelmente sairia vitorioso foi criado um novo Dataframe com os nomes e caracteristicas **mais recentes** (evitando assim repetição de nomes) dos mesmos com base em suas lutas.

E assim esperamos poder usar ao inves de colunas pré-definidas as colunas que possuem maior impacto na probabilidade de alguem vencer e implementar funcionalmente esse mecanismo.

In [ ]:
#criando um novo dataframe com apenas os lutadores sem repetição
def fighter_clean(data):
    newdata = pd.DataFrame()
    namelist = []

    for n in range(len(data.Fighter)):
        if data.loc[n,:]["Fighter"] not in namelist:
            namelist.append(data.loc[n,:]["Fighter"])
            newdata = pd.concat([newdata,data.loc[n,:]], axis=1, join='outer')

    newdata = newdata.transpose()
    return newdata


In [ ]:
# Função que separa todos os atributos do red e blue
def colunas_RB(df):
    coluna_R = ['weight_class']
    coluna_B = ['weight_class']
    for e in df.columns:
        if e != 'Referee':
            for i in range(0,1):
                if e[i] == 'R':
                    coluna_R.append(e)
                elif e[i] == 'B':
                    coluna_B.append(e)
    return coluna_R,coluna_B

In [ ]:
#criando um novo dataframe
lista_locB = colunas_RB(data)[1]
lista_locR = colunas_RB(data)[0]
data_red = data.loc[: , lista_locR]
data_blue = data.loc[: , lista_locB]

data_blue.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]
data_red.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]

data_blue = fighter_clean(data_blue)
data_red = fighter_clean(data_red)

dataRB = pd.concat([data_red,data_blue], axis=0, join='outer')

dataRB.reset_index(inplace=True)
dataRB = fighter_clean(dataRB).loc[: , ["Age","Fighter","Height_cms","Reach_cms","Weight_class","Weight_lbs"]]

dataRB = dataRB.set_index("Fighter")

dataRB = dataRB.dropna(inplace=True)

In [ ]:
dataRB.loc["Henry Cejudo" , :]

___
## Conclusão Final:

Entre os 6 modelos feitos (3 de Random Forest e 3 de Regressão Logística) melhor modelo feito foi a segunda iteração de Random Forest, em que foi obtido uma média de 68% de acertos. Levando em consideração que 67.46% das lutas do UFC foram vencidas por lutadores do lado vermelho, se o modelo apenas afirmasse que o lado vermelho sempre é o vitorioso a acurácia seria quase a mesma, o modelo não tem um bom encaixe.

Para melhorar o modelo usado poderia-se separar os dados por classe de peso, não necessariamente o mesmo fator tem o mesmo nível de importância para o peso pesado (até 120 kg) e para o peso mosca (até 56 kg), sexo, mulheres tendem a ter um estilo de luta diferente que homens, ou até faixa de tempo, o MMA mudou muito desde a criação do UFC(1993).

Mas é necessário levar em consideração que por ser um esporte de competição que quer que seus fãs não saibam o resultado para ficarem mais engajados emocionalmente nos embates , o UFC sempre põe lutadores de níveis técnicos muito próximos para não ter um atléta que será  visto como o claro vencedor do embate, assim tendo uma luta "menos emocionante", sendo considerada mais entediante, por exemplo a luta de um campeão da categoria e um novato. Assim fazer um preditor que terá como resultado o vencedor da luta não é muito possível de ser feito.

___
## Referências

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como funciona o Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)

[Referencia do Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

